In [43]:
import os
print(os.getcwd())
os.chdir('d:\\vscode_machineLearning\\internship\\Customer-Churn-Prediction')
print(os.getcwd())

d:\vscode_machineLearning\internship\Customer-Churn-Prediction
d:\vscode_machineLearning\internship\Customer-Churn-Prediction


In [44]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [45]:
df = pd.read_csv(f'artifacts/raw_data/customer_churn_removed_col.csv')

In [46]:
df.isnull().sum()

Age                           0
Gender                        0
Location                      0
Subscription_Length_Months    0
Monthly_Bill                  0
Total_Usage_GB                0
Churn                         0
dtype: int64

In [47]:
df['Gender']=df['Gender'].replace({'Male':0,'Female':1})

In [48]:
X = df.drop(columns='Churn')
y = df['Churn']

In [49]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import (OneHotEncoder,
                                   MinMaxScaler)
import numpy as np
from sklearn.model_selection import train_test_split

In [50]:
X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.20, random_state=42)

In [51]:
df['Gender']=df['Gender'].replace({'Male':0,'Female':1})

In [52]:
X_train

,Age,Gender,Location,Subscription_Length_Months,Monthly_Bill,Total_Usage_GB
75220,54,1,New York,5,84.50,205
48955,28,0,New York,24,82.06,239
44966,57,0,Chicago,12,52.29,62
13568,19,0,Houston,19,32.57,173
92727,56,1,Miami,8,33.52,314
...,...,...,...,...,...,...
6265,35,0,Miami,21,67.33,235
54886,56,0,Chicago,13,85.40,347
76820,69,0,Houston,2,76.24,321
860,55,0,Chicago,12,89.19,315


In [53]:
preprocessing = ColumnTransformer(transformers=[
    ('OHE',OneHotEncoder(drop='first',sparse=False,dtype=np.int64),['Location']),
    # ('scaling',MinMaxScaler(),['Age', 'Subscription_Length_Months', 'Monthly_Bill', 'Total_Usage_GB'])
],remainder='passthrough')

In [54]:
X_train= preprocessing.fit_transform(X_train)
X_test = preprocessing.transform(X_test)

In [55]:
X_train.shape

(80000, 9)

## model

In [56]:
from keras.layers import BatchNormalization, Dropout, Bidirectional, LSTM, Embedding, Dense ,Input
from keras.losses import binary_crossentropy
from tensorflow import keras
from keras.callbacks import LearningRateScheduler , EarlyStopping
from keras.activations import relu , sigmoid
from keras import Sequential
from keras.initializers import he_normal

In [57]:
model = Sequential()

model.add(layer=Dense(units=228,activation=relu,kernel_initializer=he_normal))
model.add(layer=Dense(units=132,activation=relu,kernel_initializer=he_normal))
model.add(BatchNormalization())
model.add(Dense(units=128,activation=relu,kernel_initializer=he_normal))
model.add(Dense(units=64,activation=relu,kernel_initializer=he_normal))
model.add(Dense(units=1,activation=sigmoid,name='output_layer'))

In [58]:
def lr_schedule(epoch, lr):
    if epoch < 1:
        return lr
    else:
        return lr * np.exp(-0.1)

# learning rate scheduler callback to descrese the learning rate gradually as the epochs increases So that my alogrithm could not jump out of Global minima.
lr_scheduler = LearningRateScheduler(lr_schedule)

# Early stopping to stop the Neural Network when we get same Validation accuracy
early_stopping = EarlyStopping(
    monitor="accuracy",
    min_delta=0.00001,
    patience=5,
    verbose=1,
    mode="auto",
    baseline=None,
    restore_best_weights=False
)

In [59]:
optimizer = keras.optimizers.Adam(learning_rate=0.0005)

In [60]:
model.compile(optimizer=optimizer, 
               loss=binary_crossentropy, 
                 metrics=['accuracy']) 

In [61]:
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20,
                    batch_size=64, 
                      callbacks=[lr_scheduler, early_stopping]) 

Epoch 1/20
